## Acquiring the data from the GalaxyZoo site

In [4]:
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.coordinates import Distance, Angle
from astropy.table import Table, Column
from astropy.io import ascii
import xml.etree.ElementTree as ET, urllib.request, gzip, io

Galaxy Zoo data sets are [here:](https://data.galaxyzoo.org/) We will use table 2. However we will need to join this catalog with the SDSS catalogs to get the galaxies' redshift values so that we can plot them in 3D. We can do that on [SciServer](http://www.sciserver.org/) by performing the following query:

    SELECT gz.ra,gz.dec,sp.z,gz.p_el,gz.p_cw,gz.p_mg
    FROM DR10.zooSpec as gz JOIN  DR8.specObjAll as sp on sp.specobjid=gz.specobjid
    WHERE sp.zWarning=0 and sp.z >0.0005   

In [20]:
#Let's read in the table we return
GZtable = ascii.read('data/galaxyZoo.csv')

In [21]:
# We'll calculate a new column with comoving distance based on the redshift
from astropy.cosmology import Planck15
GZtable.add_column(Column(Planck15.comoving_distance(GZtable['z']),name='dist'))
GZtable.add_column(Column(GZtable['ra']/15.,name='ra_h'))
GZtable.remove_column('ra')


In [22]:
# Write VO Table
GZtable.write('data/GZ.xml',format='votable')

In [23]:
# Write the xml file that contains column descriptions
import xml.etree.cElementTree as ET

root = ET.Element("ParticleTable")
doc = ET.SubElement(root, "Columns")
ET.SubElement(doc, "AddField", data="ra_h", type="RA")
ET.SubElement(doc, "AddField", data="dec", type="DEC")
ET.SubElement(doc, "AddField", data="dist", type="DIST")
ET.SubElement(doc, "AddField", data="p_el")
ET.SubElement(doc, "AddField", data="p_cw")
ET.SubElement(doc, "AddField", data="p_mg")
tree = ET.ElementTree(root)
tree.write("data/galaxyZoo_fields.xml")

### A simple red/blue diverging colormap for showing the classifications

In [2]:
cmapFile=open('data/RedBlue.cmap', 'w')
cmapFile.write('''3
0.0 0.0 1.0 1.0
1.0 1.0 1.0 1.0
1.0 0.0 0.0 1.0
''')
cmapFile.close()